In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [ ]:
BASE_URL = "https://www.booking.com/"

In [ ]:
def land_page(url):
    driver.get(url)

In [ ]:
def close_info_box():
    close_btn = driver.find_element(By.CSS_SELECTOR,
        'button[aria-label="Dismiss sign-in info."]'
    )
    close_btn.click()

In [ ]:
def toggle_currency_button():
    currency_btn = driver.find_element(By.CSS_SELECTOR,
        'button[data-testid="header-currency-picker-trigger"]'
    )
    currency_btn.click()

In [ ]:
def change_currency(currency='USD'):
    curreny_elems = driver.find_elements(By.CSS_SELECTOR,
        'button[data-testid="selection-item"]'
    )
    required_elem = None
    for elem in curreny_elems:
        cur_elem = elem.find_element(By.CLASS_NAME, "ea1163d21f")
        if cur_elem.text == currency:
            required_elem = elem
            break
    required_elem.click()

In [ ]:
def set_date(check_in, check_out):
    date_table = driver.find_element(By.CSS_SELECTOR, 'table[class="aadb8ed6d3"]')
    st_date = date_table.find_element(By.CSS_SELECTOR, f'span[data-date="{check_in}"]')
    time.sleep(1)
    end_date = date_table.find_element(By.CSS_SELECTOR, f'span[data-date="{check_out}"]')
    st_date.click()
    end_date.click()

In [ ]:
def write_desitination(destination):
    location_elem = driver.find_element(By.CSS_SELECTOR, 'input[name="ss"]')
    location_elem.send_keys(destination)
    time.sleep(2)
    options = driver.find_element(By.CSS_SELECTOR, 'ul[data-testid="autocomplete-results"]')
    final_dest = options.find_element(By.CLASS_NAME, 'a80e7dc237')
    final_dest.click()

In [ ]:
def define_people_and_room(adults=1, children=1, rooms=1):
    occupancy_btn = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="occupancy-config"]')
    occupancy_btn.click()
    range_elems = driver.find_elements(By.CLASS_NAME, 'b2b5147b20')
    adults_selector = range_elems[0]
    children_selector = range_elems[1]
    rooms_selector = range_elems[2]
    set_objects(adults_selector, adults, 'group_adults')
    # set_objects(children_selector, children)
    set_objects(rooms_selector, rooms, 'no_rooms')    

In [ ]:
def set_objects(_object, target, id):
    dec = _object.find_elements(By.CSS_SELECTOR, 'button[tabindex="-1"]')[0]
    inc = _object.find_elements(By.CSS_SELECTOR, 'button[tabindex="-1"]')[1]
    value = _object.find_element(By.ID, id).get_attribute("aria-valuenow")
    while int(value) != 1:
        dec.click()
        value = _object.find_element(By.ID, id).get_attribute("aria-valuenow")
    while int(value) != target:
        inc.click()
        value = _object.find_element(By.ID, id).get_attribute("aria-valuenow")

In [ ]:
def property_type(ht_id):
    property_type_selector = driver.find_element(By.CSS_SELECTOR, 'div[data-filters-group="ht_id"]')
    options = property_type_selector.find_elements(By.CLASS_NAME, 'f49aa20e67')
    for option in options:
        name = option.get_attribute('name')
        if str(name) == f'ht_id={ht_id}':
            option.click()
            break

In [ ]:
def property_rating(rating):
    property_rating_selector = driver.find_element(By.CSS_SELECTOR, 'div[data-filters-group="class"]')
    options = property_rating_selector.find_elements(By.CLASS_NAME, 'f49aa20e67')
    for option in options:
        name = option.get_attribute('name')
        if str(name) == f'class={rating}':
            option.click()
            break

In [ ]:
def submit():
    btn = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
    btn.click()

In [ ]:
def wait_for_next_page_load(timeout=20):
    # Wait for the presence of an element on the next page after submit
    wait = WebDriverWait(driver, timeout)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="property-card"]')))
    print("Next page has been fully loaded and is ready for interactions.")


In [ ]:
def get_data():
    data = []
    cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="property-card"]')
    for card in cards:
        try:
            title = card.find_element(By.CSS_SELECTOR, 'div[data-testid="title"]').text
            address = card.find_element(By.CSS_SELECTOR, 'span[data-testid="address"]').text
            price = card.find_element(By.CSS_SELECTOR, 'span[data-testid="price-and-discounted-price"]').text
            data.append([title, address, price])
        except NoSuchElementException as e:
            # Handle the case when an element is not found in the card
            print(f"Element not found in card. Error: {e}")
            # You can choose to skip this card or take other appropriate actions

    return data

In [ ]:
dest = "country"
check_in = '2023-08-07'
check_out = '2023-08-27'
currency = 'EUR'
adults = 10
rooms = 2
rating = 5

In [ ]:

land_page(BASE_URL)
close_info_box()
toggle_currency_button()
change_currency(currency)
write_desitination(dest)
set_date(check_in, check_out)
define_people_and_room(adults, 0, rooms)
submit()
wait_for_next_page_load()
property_type(204)
property_rating(rating)
time.sleep(5)
data = get_data()
Hotels_data = {"title": [n[0] for n in data], "Address": [n[1] for n in data], "Price": [n[2] for n in data]}
df = pd.DataFrame(Hotels_data)
df

In [ ]:
df.to_csv(dest + " " + check_in + " to " + check_out + ".csv")